<a href="https://colab.research.google.com/github/Panperception/QKD/blob/main/QKD_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI-Assisted QKD: Enhancing Security & Efficiency – Python Simulation Guide
This project will focus on integrating Machine Learning (ML) with Quantum Key Distribution (QKD) to improve security, efficiency, and error correction. The implementation will involve:

Simulating a QKD system (e.g., BB84 protocol).
Generating a quantum channel with noise to model realistic conditions.
Applying AI/ML models to enhance security, key reconciliation, and error correction.

### Initialization:

In [2]:
# %pip uninstall qiskit
!pip install pyqmc
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-algorithms
!pip install qiskit-nature
!pip install qutip
!pip install ase
!pip install scipy
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
# import numpy as np
# from qiskit import QuantumCircuit, execute
# import qiskit_aer as Aer
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator, Aer

from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature import settings
from qiskit.primitives import Sampler, StatevectorSampler
from qiskit_algorithms import HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP, SPSA, QNSPSA
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal
from functools import partial as apply_variation
from qiskit.circuit.library import Initialize
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.operators import ElectronicIntegrals
from pyscf import gto, scf
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
from qiskit_nature.second_q.drivers import PySCFDriver

import numpy as np
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
import os.path
#import pyqmc.api as pyq
import pyqmc.api as pyq
import h5py
import cmath
import math
import scipy.stats as stats
import qutip
import time, datetime
import pandas as pd


## 1. Simulation Setup in Python
We will use the following libraries:
1. Qiskit for quantum state preparation and measurement.
2. NumPy & SciPy for noise modeling and data processing.
3. TensorFlow/PyTorch for machine learning models.
4. Gym (optional) for reinforcement learning (RL).


### Step 1: Simulating QKD (BB84 Protocol)
BB84 is a widely used QKD protocol where Alice sends qubits in random bases, and Bob measures them in a matching or mismatching basis.

Basic BB84 Implementation:

In [4]:
def generate_bb84_bits(n):
    """Generate random bit string and random bases for Alice"""
    bits = np.random.randint(0, 2, n)
    bases = np.random.randint(0, 2, n)  # 0 = Z basis, 1 = X basis
    return bits, bases

def prepare_qubits(bits, bases):
    """Create quantum states based on Alice's bits and bases"""
    qc_list = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if basis == 1:
            qc.h(0)  # Apply Hadamard if in X basis
        if bit == 1:
            qc.x(0)  # Apply X gate if bit is 1
        qc_list.append(qc)
    return qc_list

def measure_qubits(qc_list, bases):
    """Bob measures the qubits"""
    results = []
    backend = Aer.get_backend('aer_simulator')
    for qc, basis in zip(qc_list, bases):
        if basis == 1:
            qc.h(0)  # Apply Hadamard back if measuring in X basis
        qc.measure(0, 0)
        result = backend.run(qc, shots=10).result()
        measured_bit = int(list(result.get_counts().keys())[0])  # Extract bit
        results.append(measured_bit)
    return results

# Simulate BB84
n = 100
alice_bits, alice_bases = generate_bb84_bits(n)
qc_list = prepare_qubits(alice_bits, alice_bases)
bob_bases = np.random.randint(0, 2, n)
bob_measurements = measure_qubits(qc_list, bob_bases)

## 2. AI-Assisted Attack Detection in Noisy Quantum Channels
### Step 2: Simulating a Quantum Channel with Noise
We simulate an intercept-resend attack or decoherence noise using depolarizing channels.

In [5]:
from qiskit_aer.noise import NoiseModel, errors



In [6]:
def create_noise_model():
    """Define a depolarizing noise model to simulate an eavesdropper"""
    noise_model = NoiseModel()
    error = errors.depolarizing_error(0.05, 1)  # 5% depolarization probability
    noise_model.add_all_qubit_quantum_error(error, ['measure'])
    return noise_model

def simulate_noisy_bb84(qc_list, bases):
    """Simulate Bob's measurement with a noisy quantum channel"""
    results = []
    backend = Aer.get_backend('aer_simulator')
    noise_model = create_noise_model()
    for qc, basis in zip(qc_list, bases):
        if basis == 1:
            qc.h(0)
        qc.measure(0, 0)
        result = backend.run(qc, noise_model=noise_model, shots=10).result()
        measured_bit = int(list(result.get_counts().keys())[0])
        results.append(measured_bit)
    return results

bob_noisy_measurements = simulate_noisy_bb84(qc_list, bob_bases)


## 3. Using Machine Learning for Attack Detection
An AI model can analyze deviations in the quantum channel noise to detect potential eavesdropping.

### Step 3: Training an ML Model for Attack Detection
We will train a Neural Network (NN) to differentiate between normal and attack scenarios based on the quantum bit error rate (QBER).

Feature Engineering

Input: Alice’s and Bob’s measurements, bit mismatch rates.

Output: Probability of an eavesdropping attack.

Data Preparation

#### Data Preparation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Generate dataset
def generate_qkd_dataset(samples=1000):
    data = []
    for _ in range(samples):
        # Simulate normal transmission
        alice_bits, alice_bases = generate_bb84_bits(100)
        qc_list = prepare_qubits(alice_bits, alice_bases)
        bob_bases = np.random.randint(0, 2, 100)
        bob_measurements = measure_qubits(qc_list, bob_bases)

        # Simulate attack/noise
        if np.random.rand() < 0.5:  # 50% chance of attack
            bob_measurements = simulate_noisy_bb84(qc_list, bob_bases)
            label = 1  # Attack
        else:
            label = 0  # No Attack

        qber = np.sum(np.array(alice_bits) != np.array(bob_measurements)) / 100
        data.append([qber, label])

    return pd.DataFrame(data, columns=['QBER', 'Attack'])

df = generate_qkd_dataset()
X_train, X_test, y_train, y_test = train_test_split(df[['QBER']], df['Attack'], test_size=0.2)


#### Random Forest

In [ ]:
# Train an AI Model (Random Forest Classifier)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print("Model Train Accuracy:", clf.score(X_train, y_train))
# Test AI Model
print("Model Accuracy:", clf.score(X_test, y_test))


#### Deep NN - Fully Connected

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Build the deep learning model using Keras
model = Sequential([
    Dense(32, activation='leakyrelu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=50, validation_split=0.2, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", accuracy)

# Making predictions on test data
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Sample predictions:", y_pred[:10].flatten())

In [28]:
!pip install keras-tuner --upgrade

ERROR: unknown command "keras_tuner"


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras_tuner as kt

def build_model(hp):
    model = Sequential()
    # Input layer with tunable number of neurons
    model.add(Dense(units=hp.Int('units_input', min_value=32, max_value=256, step=32),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout_input', min_value=0.0, max_value=0.5, step=0.1)))

    # Add 1 to 3 hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=256, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(
                      hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=50,
    executions_per_trial=3,
    directory='qkd_tuning',
    project_name='attack_detection'
)

tuner.search(X_train, y_train, epochs=50, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate on the test set
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print("Best model test accuracy:", test_acc)


Trial 3 Complete [00h 00m 37s]
val_accuracy: 0.7854166626930237

Best val_accuracy So Far: 0.8187500238418579
Total elapsed time: 00h 02m 04s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
32                |192               |units_input
0.4               |0.3               |dropout_input
3                 |3                 |num_layers
64                |32                |units_0
0.2               |0.2               |dropout_0
0.01              |0.001             |learning_rate
128               |32                |units_1
0                 |0                 |dropout_1
192               |32                |units_2
0                 |0                 |dropout_2

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.4627 - loss: 0.6980 - val_accuracy: 0.7812 - val_loss: 0.6627
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6427 - loss: 0.6338 - val_accuracy: 0.7312 - val_loss: 0.5621
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6731 - loss: 0.5882 - val_accuracy: 0.8125 - val_loss: 0.4823
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7399 - loss: 0.5276 - val_accuracy: 0.7750 - val_loss: 0.4963
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7699 - loss: 0.4833 - val_accuracy: 0.8125 - val_loss: 0.4522
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7927 - loss: 0.4848 - val_accuracy: 0.8188 - val_loss: 0.4351
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7119 - loss: 0.5488 - val_accuracy: 0.8000 - val_loss: 0.4605
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7115 - loss: 0.5351 - val_accuracy: 0.8188 - val_loss: 0.4244
Ep

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.4827 - loss: 0.6947 - val_accuracy: 0.4812 - val_loss: 0.6896
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5576 - loss: 0.6823 - val_accuracy: 0.6375 - val_loss: 0.6380
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6424 - loss: 0.6358 - val_accuracy: 0.7812 - val_loss: 0.5002
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7316 - loss: 0.5327 - val_accuracy: 0.7500 - val_loss: 0.5270
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7820 - loss: 0.4915 - val_accuracy: 0.7812 - val_loss: 0.4468
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7418 - loss: 0.5251 - val_accuracy: 0.8188 - val_loss: 0.4495
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7782 - loss: 0.4610 - val_accuracy: 0.8125 - val_loss: 0.4442
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7631 - loss: 0.5048 - val_accuracy: 0.7812 - val_loss: 0.4382
Ep

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.5234 - loss: 0.6961 - val_accuracy: 0.8188 - val_loss: 0.6621
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6229 - loss: 0.6537 - val_accuracy: 0.8188 - val_loss: 0.4540
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7472 - loss: 0.5414 - val_accuracy: 0.7937 - val_loss: 0.4822
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7880 - loss: 0.4983 - val_accuracy: 0.8000 - val_loss: 0.4236
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7990 - loss: 0.4588 - val_accuracy: 0.8125 - val_loss: 0.4800
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7810 - loss: 0.4929 - val_accuracy: 0.8188 - val_loss: 0.4469
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7143 - loss: 0.5564 - val_accuracy: 0.8125 - val_loss: 0.4789
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7877 - loss: 0.4727 - val_accuracy: 0.8188 - val_loss: 0.4228
Ep

KeyboardInterrupt: 

In [35]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# ------------------------------
# 2. Build the LSTM Model
# ------------------------------
model = Sequential()
# First LSTM layer with return_sequences=True to feed the next LSTM layer.
model.add(LSTM(64, input_shape=(timesteps, num_features), return_sequences=True))
model.add(Dropout(0.2))
# Second LSTM layer that outputs a fixed-size vector.
model.add(LSTM(32))
model.add(Dropout(0.2))
# Output layer for binary classification.
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# ------------------------------
# 3. Train the Model
# ------------------------------
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# ------------------------------
# 4. Evaluate the Model
# ------------------------------
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 64)              │          19,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 50, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,649 (123.63 KB)

 Trainable params: 31,649 (123.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7896 - loss: 0.5253 - val_accuracy: 0.9656 - val_loss: 0.1222
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9674 - loss: 0.1225 - val_accuracy: 0.9656 - val_loss: 0.1138
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9699 - loss: 0.0957 - val_accuracy: 0.9656 - val_loss: 0.0886
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9826 - loss: 0.0738 - val_accuracy: 0.9688 - val_loss: 0.0757
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9813 - loss: 0.0705 - val_accuracy: 0.9563 - val_loss: 0.1268
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9783 - loss: 0.0660 - val_accuracy: 0.9719 - val_loss: 0.0678
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9894 - loss: 0.0458 - val_accuracy: 0.9625 - val_loss: 0.0890
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9902 - loss: 0.0372 - val_accuracy: 0.9750 - va

In [37]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, GlobalAveragePooling1D
from tensorflow.keras.models import Model

# Positional encoding function
def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    angle_rads = pos * angle_rates
    pos_encoding = np.zeros(angle_rads.shape)
    pos_encoding[:, 0::2] = np.sin(angle_rads[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = pos_encoding[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

# Transformer encoder block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):
    # Layer normalization and multi-head attention
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = Add()([x, inputs])

    # Feed-forward network
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return Add()([x, res])

# Define model parameters
seq_len = 50         # Number of timesteps in each sample
num_features = 10    # Number of features per timestep
d_model = 64         # Model dimension

# Input layer
inputs = Input(shape=(seq_len, num_features))
# Project inputs to the desired dimension
x = Dense(d_model)(inputs)

# Add positional encoding
x += positional_encoding(seq_len, d_model)

# Stack transformer encoder blocks
num_transformer_blocks = 2
for _ in range(num_transformer_blocks):
    x = transformer_encoder(x, head_size=32, num_heads=4, ff_dim=64, dropout=0.1)

# Global pooling and output layer
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation="sigmoid")(x)

# Build and compile the model
model = Model(inputs, outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 50, 10)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 50, 64)         │            704 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 50, 64)         │              0 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 50, 64)         │            128 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 50, 64)         │         33,216 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 50, 64)         │              0 │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 50, 64)         │              0 │ dropout_7[0][0],       │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 50, 64)         │            128 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 50, 64)         │          4,160 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 50, 64)         │              0 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 50, 64)         │          4,160 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 50, 64)         │              0 │ dense_8[0][0],         │
│                           │                        │                │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 50, 64)         │            128 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 50, 64)         │         33,216 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 50, 64)         │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_3 (Add)          

 Total params: 84,353 (329.50 KB)

 Trainable params: 84,353 (329.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - accuracy: 0.4921 - loss: 0.7908 - val_accuracy: 0.5406 - val_loss: 0.7819
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4889 - loss: 0.7487 - val_accuracy: 0.5375 - val_loss: 0.6973
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5137 - loss: 0.7169 - val_accuracy: 0.4594 - val_loss: 0.7196
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5044 - loss: 0.7224 - val_accuracy: 0.5406 - val_loss: 0.7073
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4833 - loss: 0.7155 - val_accuracy: 0.4437 - val_loss: 0.7035
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4966 - loss: 0.7037 - val_accuracy: 0.4625 - val_loss: 0.7018
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5200 - loss: 0.7001 - val_accuracy: 0.4563 - val_loss: 0.7056
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5339 - loss: 0.6921 - val_accuracy: 0.4563 - val_los

## 4. Reinforcement Learning for Dynamic Key Rate Allocation
Instead of fixed QKD key rates, we can use reinforcement learning (RL) to optimize transmission rates based on channel conditions.

Approach:

Define the state as channel conditions and noise levels.
Define actions as key rate adjustments.
Use Q-learning to maximize secure key generation.
### Step 4: RL-Based Key Rate Allocation

In [11]:
import gym
import numpy as np

class QKDSimulator(gym.Env):
    def __init__(self):
        self.state = 0.5  # QBER as state
        self.action_space = np.array([0.1, 0.2, 0.3])  # Adjust key rate
        self.reward = 0

    def step(self, action):
        self.state += np.random.normal(0, 0.01)  # Simulate noise variation
        self.reward = -abs(self.state - action)  # Reward based on key rate efficiency
        return self.state, self.reward

    def reset(self):
        self.state = 0.5
        return self.state

env = QKDSimulator()
for _ in range(10):
    action = np.random.choice(env.action_space)
    state, reward = env.step(action)
    print(f"Action: {action}, New QBER: {state:.3f}, Reward: {reward:.3f}")


Action: 0.2, New QBER: 0.503, Reward: -0.303
Action: 0.2, New QBER: 0.498, Reward: -0.298
Action: 0.3, New QBER: 0.498, Reward: -0.198
Action: 0.3, New QBER: 0.493, Reward: -0.193
Action: 0.2, New QBER: 0.486, Reward: -0.286
Action: 0.2, New QBER: 0.480, Reward: -0.280
Action: 0.1, New QBER: 0.473, Reward: -0.373
Action: 0.2, New QBER: 0.470, Reward: -0.270
Action: 0.1, New QBER: 0.467, Reward: -0.367
Action: 0.2, New QBER: 0.457, Reward: -0.257


## Conclusion & Next Steps
This project integrates AI into QKD for attack detection, error correction, and dynamic key rate optimization.
Possible improvements:
* Implement deep learning models for attack detection.
* Test on real quantum hardware using IBMQ.
* Extend reinforcement learning for network-wide QKD optimization.
